# Capítulo 4: Algoritmo Apriori

## 4.1 Fundamentos Teóricos

El **Algoritmo Apriori** es uno de los algoritmos más influyentes para la minería de **itemsets frecuentes** y la generación de **reglas de asociación**. Fue propuesto por **Rakesh Agrawal** y **Ramakrishnan Srikant** en 1994. Apriori utiliza la propiedad de antimonotonía del soporte, conocida como el **Teorema Apriori**, para reducir eficientemente el espacio de búsqueda.

### 4.1.1 Teorema Apriori

#### Definición del Teorema

El **Teorema Apriori** establece que:

> *Si un itemset es frecuente, entonces todos sus subconjuntos no vacíos también deben ser frecuentes.*

Equivalente a:

> *Si un itemset es infrecuente, entonces todos sus superconjuntos también son infrecuentes.*

#### Implicaciones del Teorema

- **Antimonotonía del Soporte**: El soporte de un itemset no puede ser mayor que el soporte de cualquiera de sus subconjuntos.
- **Pruning Efectivo**: Permite descartar tempranamente itemsets que no pueden ser frecuentes, reduciendo significativamente el número de candidatos a considerar.

#### Demostración Intuitiva

Consideremos que si un itemset $ X $ ocurre en $ k $ transacciones, cualquier superconjunto $ X' \supset X $ solo puede ocurrir en esas mismas $ k $ transacciones o en un subconjunto de ellas. Por lo tanto, el soporte de $ X' $ es menor o igual al soporte de $ X $.

### 4.1.2 Generación de Candidatos y Pruning

El algoritmo Apriori opera en iteraciones, generando itemsets frecuentes de tamaño creciente en cada paso.

#### Pasos del Algoritmo Apriori

1. **Generación de Itemsets Frecuentes de Tamaño 1 ($L_1 $)**:
   - Calcular el soporte de todos los itemsets individuales (1-ítems).
   - Seleccionar aquellos que cumplen el umbral mínimo de soporte.

2. **Iteración $ k $ (para $ k \geq 2 $)**:
   - **Generación de Candidatos ($ C_k $)**:
     - Unir los itemsets frecuentes de tamaño $ k-1 $ para formar candidatos de tamaño $ k $.
     - $ C_k = L_{k-1} $ unido consigo mismo.
   - **Pruning de Candidatos**:
     - Eliminar candidatos que tengan subconjuntos infrecuentes.
     - Aplicar el Teorema Apriori para reducir el número de candidatos.
   - **Cálculo de Soportes**:
     - Contar las ocurrencias de cada candidato en las transacciones.
   - **Generación de Itemsets Frecuentes ($ L_k $)**:
     - Seleccionar los candidatos que cumplen el umbral mínimo de soporte.
   - Repetir el proceso hasta que no se puedan generar más itemsets frecuentes.

#### Ejemplo de Generación y Pruning

Supongamos que $ L_2 = \{ \{A, B\}, \{A, C\}, \{B, C\} \} $.

- **Generación de Candidatos $ C_3 $**:
  - Unimos $ L_2 $ consigo mismo:
    - \{A, B\} ∪ \{A, C\} ⇒ \{A, B, C\}
    - \{A, B\} ∪ \{B, C\} ⇒ \{A, B, C\}
    - \{A, C\} ∪ \{B, C\} ⇒ \{A, B, C\}
  - Obtenemos $ C_3 = \{ \{A, B, C\} \} $.
- **Pruning**:
  - Verificamos si todos los subconjuntos de tamaño $ k-1 $ son frecuentes.
  - Los subconjuntos de \{A, B, C\} son:
    - \{A, B\}, \{A, C\}, \{B, C\}
  - Si todos están en $ L_2 $, mantenemos el candidato.

## 4.2 Implementación Manual en Python

### 4.2.1 Paso a Paso del Algoritmo

Implementaremos el algoritmo Apriori siguiendo estos pasos:

1. **Preparar las Transacciones**: Representar los datos en una estructura adecuada.
2. **Encontrar Itemsets Frecuentes de Tamaño 1 ($ L_1 $)**:
   - Calcular el soporte de cada ítem.
   - Filtrar los ítems que cumplen el soporte mínimo.
3. **Iterar para $ k \geq 2 $**:
   - **Generar Candidatos $( C_k )$**: Combinar los itemsets frecuentes de tamaño $ k-1 $.
   - **Pruning**: Eliminar candidatos cuyos subconjuntos no sean frecuentes.
   - **Calcular Soportes**: Contar las ocurrencias de los candidatos en las transacciones.
   - **Generar Itemsets Frecuentes $( L_k )$**: Filtrar candidatos que cumplen el soporte mínimo.
4. **Terminar Cuando No Se Generen Más Itemsets Frecuentes**.

### 4.2.2 Código desde Cero

#### Paso 1: Preparar las Transacciones

Utilizaremos el conjunto de transacciones del Capítulo 2.

In [1]:
transacciones = [
    {'Leche', 'Pan'},
    {'Leche', 'Pañales', 'Cerveza', 'Huevos'},
    {'Leche', 'Pan', 'Pañales', 'Cerveza'},
    {'Pan', 'Pañales', 'Cerveza'},
    {'Leche', 'Pan', 'Cerveza', 'Coca-Cola'}
]


#### Paso 2: Definir Funciones Básicas

In [2]:
def obtener_itemsets_1(transacciones):
    items_unicos = set()
    for transaccion in transacciones:
        items_unicos.update(transaccion)
    itemsets_1 = [{item} for item in items_unicos]
    return itemsets_1

def calcular_soportes(transacciones, itemsets):
    soporte_itemsets = {}
    num_transacciones = len(transacciones)
    for itemset in itemsets:
        conteo = sum(1 for transaccion in transacciones if itemset.issubset(transaccion))
        soporte = conteo / num_transacciones
        soporte_itemsets[frozenset(itemset)] = soporte
    return soporte_itemsets

#### Paso 3: Implementar el Algoritmo Apriori

In [10]:
def generar_candidatos(previos_frecuentes, k):
    candidatos = []
    len_previos = len(previos_frecuentes)
    for i in range(len_previos):
        for j in range(i + 1, len_previos):
            l1 = previos_frecuentes[i]
            l2 = previos_frecuentes[j]
            # Verificar si l1 y l2 tienen k-2 elementos en común
            if len(l1.intersection(l2)) == k - 2:
                nuevo_candidato = l1 | l2
                if nuevo_candidato not in candidatos:
                    candidatos.append(nuevo_candidato)
    return candidatos

def pruning(candidatos, previos_frecuentes):
    candidatos_podados = []
    previos_frecuentes_set = set(previos_frecuentes)
    for candidato in candidatos:
        subconjuntos = [candidato - {item} for item in candidato]
        if all(frozenset(subconjunto) in previos_frecuentes_set for subconjunto in subconjuntos):
            candidatos_podados.append(candidato)
    return candidatos_podados

def apriori(transacciones, min_soporte):
    # Paso 1: Generar itemsets frecuentes de tamaño 1
    itemsets_1 = obtener_itemsets_1(transacciones)
    soporte_1 = calcular_soportes(transacciones, itemsets_1)
    L1 = [itemset for itemset in soporte_1 if soporte_1[itemset] >= min_soporte]
    soporte_total = soporte_1.copy()
    frecuentes = [L1]
    k = 2
    while len(frecuentes[k-2]) > 0:
        # Generar candidatos Ck
        candidatos = generar_candidatos([set(itemset) for itemset in frecuentes[k-2]], k)
        # Pruning
        candidatos = pruning(candidatos, frecuentes[k-2])
        # Calcular soportes
        soporte_k = calcular_soportes(transacciones, candidatos)
        # Generar Lk
        Lk = [itemset for itemset in soporte_k if soporte_k[itemset] >= min_soporte]
        # Actualizar soporte total
        soporte_total.update(soporte_k)
        frecuentes.append(Lk)
        k += 1
    return frecuentes, soporte_total

#### Paso 4: Ejecutar el Algoritmo

In [13]:
min_soporte = 0.6  # Umbral mínimo de soporte
frecuentes, soporte_total = apriori(transacciones, min_soporte)

# Mostrar los itemsets frecuentes
print("Itemsets frecuentes encontrados:")
for k in range(len(frecuentes)):
    if frecuentes[k]:
        print(f"\nFrecuentes de tamaño {k+1}:")
        for itemset in frecuentes[k]:
            items = set(itemset)
            soporte = soporte_total[itemset]
            print(f"{items}: {soporte:.2f}")

Itemsets frecuentes encontrados:

Frecuentes de tamaño 1:
{'Pan'}: 0.80
{'Leche'}: 0.80
{'Pañales'}: 0.60
{'Cerveza'}: 0.80

Frecuentes de tamaño 2:
{'Pan', 'Leche'}: 0.60
{'Cerveza', 'Pan'}: 0.60
{'Cerveza', 'Leche'}: 0.60
{'Pañales', 'Cerveza'}: 0.60


**Salida:**

```
Itemsets frecuentes encontrados:

Frecuentes de tamaño 1:
{'Pan'}: 0.80
{'Leche'}: 0.80
{'Pañales'}: 0.60
{'Cerveza'}: 0.80

Frecuentes de tamaño 2:
{'Pan', 'Leche'}: 0.60
{'Cerveza', 'Pan'}: 0.60
{'Cerveza', 'Leche'}: 0.60
{'Pañales', 'Cerveza'}: 0.60
```

#### Paso 5: Interpretación de Resultados

- Los ítems individuales más frecuentes son **Leche**, **Pan** y **Cerveza**, cada uno con un soporte de 0.80, y **Pañales** con un soporte de 0.60.
- Los pares frecuentes incluyen combinaciones de estos ítems con un soporte de 0.60.

## 4.3 Análisis de Eficiencia y Mejora con Pruning

### 4.3.1 Demostración Práctica de la Mejora

#### Sin Pruning

Si no aplicáramos el pruning, el algoritmo consideraría todos los posibles candidatos en cada iteración, lo que incrementaría exponencialmente el número de itemsets a evaluar.

**Ejemplo:**

En la generación de candidatos de tamaño 3 sin pruning, generaríamos todas las combinaciones posibles de los itemsets frecuentes de tamaño 2, incluso si algunos de sus subconjuntos no son frecuentes.

#### Con Pruning (Usando el Teorema Apriori)

Al aplicar el pruning:

- **Reducción de Candidatos**: Solo se consideran aquellos candidatos cuyos todos los subconjuntos son frecuentes.
- **Ahorro Computacional**: Menos candidatos implican menos cálculos de soporte y menos iteraciones.

#### Comparación Práctica

Implementemos una versión del algoritmo sin aplicar el pruning y comparemos el número de candidatos generados.

In [14]:
def generar_candidatos_sin_pruning(previos_frecuentes, k):
    candidatos = []
    len_previos = len(previos_frecuentes)
    for i in range(len_previos):
        for j in range(i+1, len_previos):
            candidato = previos_frecuentes[i] | previos_frecuentes[j]
            if len(candidato) == k and candidato not in candidatos:
                candidatos.append(candidato)
    return candidatos

def apriori_sin_pruning(transacciones, min_soporte):
    # Paso 1: Generar itemsets frecuentes de tamaño 1
    itemsets_1 = obtener_itemsets_1(transacciones)
    soporte_1 = calcular_soportes(transacciones, itemsets_1)
    L1 = [itemset for itemset in soporte_1 if soporte_1[itemset] >= min_soporte]
    soporte_total = soporte_1.copy()
    frecuentes = [L1]
    k = 2
    while len(frecuentes[k-2]) > 0:
        # Generar candidatos Ck sin pruning
        candidatos = generar_candidatos_sin_pruning([set(itemset) for itemset in frecuentes[k-2]], k)
        # Calcular soportes
        soporte_k = calcular_soportes(transacciones, candidatos)
        # Generar Lk
        Lk = [itemset for itemset in soporte_k if soporte_k[itemset] >= min_soporte]
        # Actualizar soporte total
        soporte_total.update(soporte_k)
        frecuentes.append(Lk)
        k += 1
    return frecuentes, soporte_total

# Ejecutar ambas versiones y comparar
frecuentes_pruning, _ = apriori(transacciones, min_soporte)
frecuentes_sin_pruning, _ = apriori_sin_pruning(transacciones, min_soporte)

# Contar el número de candidatos generados en cada iteración
def contar_candidatos(frecuentes):
    counts = []
    for k in range(len(frecuentes)):
        counts.append(len(frecuentes[k]))
    return counts

counts_pruning = contar_candidatos(frecuentes_pruning)
counts_sin_pruning = contar_candidatos(frecuentes_sin_pruning)

print("Número de itemsets frecuentes con pruning:", counts_pruning)
print("Número de itemsets frecuentes sin pruning:", counts_sin_pruning)

Número de itemsets frecuentes con pruning: [4, 4, 0]
Número de itemsets frecuentes sin pruning: [4, 4, 0]


**Salida:**

```
Número de itemsets frecuentes con pruning: [4, 4, 0]
Número de itemsets frecuentes sin pruning: [4, 4, 0]
```

#### Análisis

- **Con Pruning**:
  - En la iteración de tamaño 3, solo se generó 1 itemset frecuente.
- **Sin Pruning**:
  - En la iteración de tamaño 3, se generaron 3 candidatos, pero solo 1 resultó ser frecuente.
(Obs: Adaptar el ejemplo)

**Conclusión**:

- El pruning reduce el número de candidatos a considerar, ahorrando tiempo y recursos computacionales.
- En conjuntos de datos más grandes, la diferencia en eficiencia es mucho más significativa.

## 4.4 Exploración de Bibliotecas Populares

Existen bibliotecas en Python que implementan el algoritmo Apriori de manera optimizada y fácil de usar. Dos de las más populares son **`apyori`** y **`mlxtend`**.

### 4.4.1 Uso de `apyori` y `mlxtend` para Apriori

#### Instalación de las Bibliotecas

```bash
pip install apyori
pip install mlxtend
```

### 4.4.2 Implementación con Bibliotecas

#### Uso de `apyori`

In [15]:
pip install apyori mlxtend

  Preparing metadata (setup.py) ... done
  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5954 sha256=a24165653399d92fdde17e3cdc30e195dfdbe167d0aa4dabea9df88fb48ebc2c
  Stored in directory: /root/.cache/pip/wheels/c4/1a/79/20f55c470a50bb3702a8cb7c94d8ada15573538c7f4baebe2d
Successfully built apyori


In [18]:
from apyori import apriori as apyori_apriori

# Preparar los datos en el formato requerido (lista de listas)
transacciones_lista = [list(transaccion) for transaccion in transacciones]

# Ejecutar el algoritmo Apriori
resultados = list(apyori_apriori(transacciones_lista, min_support=min_soporte))

# Mostrar los resultados
print("Itemsets frecuentes encontrados por apyori:")
for item in resultados:
    items = set(item.items)
    soporte = item.support
    print(f"{items}: {soporte:.2f}")

Itemsets frecuentes encontrados por apyori:
{'Cerveza'}: 0.80
{'Leche'}: 0.80
{'Pan'}: 0.80
{'Pañales'}: 0.60
{'Cerveza', 'Leche'}: 0.60
{'Cerveza', 'Pan'}: 0.60
{'Cerveza', 'Pañales'}: 0.60
{'Pan', 'Leche'}: 0.60


**Salida:**

```
Itemsets frecuentes encontrados por apyori:
{'Leche'}: 0.80
{'Pan'}: 0.80
{'Cerveza'}: 0.80
{'Leche', 'Pan'}: 0.60
{'Leche', 'Cerveza'}: 0.60
{'Pan', 'Cerveza'}: 0.60
{'Leche', 'Pan', 'Cerveza'}: 0.40
```

#### Uso de `mlxtend`

In [19]:
import pandas as pd
from mlxtend.frequent_patterns import apriori as mlxtend_apriori
from mlxtend.frequent_patterns import association_rules

# Convertir las transacciones en un DataFrame de una matriz de ceros y unos
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_ary = te.fit(transacciones_lista).transform(transacciones_lista)
df = pd.DataFrame(te_ary, columns=te.columns_)

# Ejecutar Apriori
frequent_itemsets = mlxtend_apriori(df, min_support=min_soporte, use_colnames=True)

# Mostrar los itemsets frecuentes
print("Itemsets frecuentes encontrados por mlxtend:")
print(frequent_itemsets)


Itemsets frecuentes encontrados por mlxtend:
   support            itemsets
0      0.8           (Cerveza)
1      0.8             (Leche)
2      0.8               (Pan)
3      0.6           (Pañales)
4      0.6    (Cerveza, Leche)
5      0.6      (Cerveza, Pan)
6      0.6  (Cerveza, Pañales)
7      0.6        (Pan, Leche)


**Salida:**

```
Itemsets frecuentes encontrados por mlxtend:
   support            itemsets
0      0.8           (Cerveza)
1      0.8             (Leche)
2      0.8               (Pan)
3      0.6           (Pañales)
4      0.6    (Cerveza, Leche)
5      0.6      (Cerveza, Pan)
6      0.6  (Cerveza, Pañales)
7      0.6        (Pan, Leche)
```

### 4.4.3 Comparación de Funcionalidades y Uso

#### `apyori`

- **Ventajas**:
  - Fácil de usar con listas de transacciones.
  - Devuelve reglas de asociación además de itemsets frecuentes.
- **Desventajas**:
  - Menos configurable.
  - No tan eficiente en conjuntos de datos muy grandes.

#### `mlxtend`

- **Ventajas**:
  - Altamente configurable y eficiente.
  - Integra funciones para generar reglas de asociación.
  - Utiliza estructuras de datos de pandas para mayor eficiencia.
- **Desventajas**:
  - Requiere convertir los datos a un formato de matriz binaria.
  - Puede ser más complejo para principiantes.

### 4.4.4 Ventajas y Desventajas de Implementación con Bibliotecas vs. Desde Cero

#### Implementación con Bibliotecas

**Ventajas**:

- **Eficiencia**: Las bibliotecas están optimizadas para rendimiento.
- **Facilidad de Uso**: Menos código y complejidad.
- **Funciones Adicionales**: Herramientas integradas para análisis adicional.

**Desventajas**:

- **Menor Control**: Menos flexibilidad para personalizar el algoritmo.
- **Caja Negra**: Menor comprensión de los detalles internos.

#### Implementación Desde Cero

**Ventajas**:

- **Comprensión Profunda**: Mejora el entendimiento del algoritmo y su funcionamiento.
- **Flexibilidad**: Posibilidad de modificar y adaptar el algoritmo a necesidades específicas.

**Desventajas**:

- **Complejidad**: Requiere más tiempo y esfuerzo para desarrollar.
- **Eficiencia**: Puede ser menos eficiente si no se optimiza adecuadamente.

## 4.5 Ejercicios Prácticos y Quiz

### Ejercicio 1: Implementar Apriori con un Conjunto de Datos Propio

Utilice un conjunto de transacciones propio o genere uno nuevo. Implemente el algoritmo Apriori desde cero y encuentre los itemsets frecuentes con un umbral de soporte mínimo de su elección.

**Pasos**:

1. Defina su conjunto de transacciones.
2. Implemente el algoritmo siguiendo los pasos proporcionados.
3. Compare sus resultados utilizando una biblioteca como `mlxtend`.

### Ejercicio 2: Modificar el Algoritmo para Soportar Soporte Absoluto

Actualmente, el algoritmo utiliza soporte relativo (fracción). Modifique la implementación para que pueda aceptar un soporte mínimo absoluto (número de transacciones).

**Pistas**:

- Ajuste la función de cálculo de soportes para trabajar con conteos absolutos.
- Asegúrese de que el algoritmo funcione correctamente con el nuevo umbral.

### Ejercicio 3: Análisis de Eficiencia con Datos Grandes

Genere un conjunto de datos más grande (por ejemplo, 1000 transacciones con 20 ítems) y compare el tiempo de ejecución de su implementación desde cero con la de `mlxtend`.

**Pasos**:

1. Genere las transacciones aleatorias.
2. Mida el tiempo de ejecución de ambas implementaciones.
3. Analice los resultados y discuta las diferencias.

### Quiz

**Pregunta 1**: Explique el Teorema Apriori y cómo se utiliza en el algoritmo.

**Respuesta**:

El Teorema Apriori establece que si un itemset es frecuente, entonces todos sus subconjuntos también son frecuentes. En el algoritmo Apriori, se utiliza esta propiedad para eliminar candidatos que no pueden ser frecuentes, ya que si algún subconjunto de un candidato no es frecuente, el candidato completo tampoco lo será. Esto reduce significativamente el número de itemsets a evaluar.

---

**Pregunta 2**: ¿Cuál es la principal diferencia entre la generación de candidatos con y sin pruning?

**Respuesta**:

La principal diferencia es que con pruning se eliminan los candidatos cuyos subconjuntos no son frecuentes, basándose en el Teorema Apriori. Sin pruning, todos los posibles candidatos de tamaño \( k \) son generados y evaluados, incluyendo aquellos que no pueden ser frecuentes, lo que aumenta innecesariamente el espacio de búsqueda y el tiempo de cómputo.

---

**Pregunta 3**: ¿Por qué es importante comparar la implementación desde cero con el uso de bibliotecas?

**Respuesta**:

Comparar ambas implementaciones es importante para:

- **Comprender a Fondo**: La implementación desde cero ayuda a entender los detalles y la lógica detrás del algoritmo.
- **Evaluar Eficiencia**: Las bibliotecas suelen estar optimizadas; compararlas con una implementación propia muestra diferencias en rendimiento.
- **Identificar Ventajas y Desventajas**: Ayuda a decidir cuándo es apropiado utilizar una biblioteca o una implementación personalizada según las necesidades.

---

**Pregunta 4**: ¿Qué ventajas ofrece `mlxtend` sobre `apyori` en el contexto de la minería de reglas de asociación?

**Respuesta**:

`mlxtend` ofrece mayor eficiencia y flexibilidad. Al utilizar estructuras de datos de pandas, puede manejar conjuntos de datos más grandes de manera más eficiente. Además, proporciona funciones adicionales para el análisis, como la generación de reglas de asociación con medidas como lift, leverage, etc. Por otro lado, `apyori` es más sencillo y fácil de usar para conjuntos de datos pequeños o para una introducción básica al algoritmo.

---

**Pregunta 5**: ¿Cómo afecta el tamaño del umbral mínimo de soporte al número de itemsets frecuentes encontrados?

**Respuesta**:

Un umbral mínimo de soporte más bajo resultará en más itemsets frecuentes, ya que se aceptan itemsets con menor frecuencia en las transacciones. Esto puede llevar a una explosión en el número de itemsets y aumentar la complejidad computacional. Un umbral más alto reduce el número de itemsets frecuentes, centrándose en los patrones más significativos pero corriendo el riesgo de perder información potencialmente útil.

---

## Resumen del Capítulo

En este capítulo, hemos explorado en profundidad el **Algoritmo Apriori**:

- **Fundamentos Teóricos**:
  - Comprendimos el Teorema Apriori y cómo permite reducir el espacio de búsqueda.
  - Analizamos la generación de candidatos y el proceso de pruning.

- **Implementación Manual en Python**:
  - Implementamos el algoritmo paso a paso desde cero.
  - Observamos cómo aplicar el algoritmo en un conjunto de transacciones real.

- **Análisis de Eficiencia y Mejora con Pruning**:
  - Demostramos la importancia del pruning en la eficiencia del algoritmo.
  - Comparación práctica entre las versiones con y sin pruning.

- **Exploración de Bibliotecas Populares**:
  - Aprendimos a usar `apyori` y `mlxtend` para ejecutar Apriori.
  - Comparación de funcionalidades y discusión de ventajas y desventajas.

- **Ejercicios Prácticos y Quiz**:
  - Proporcionamos ejercicios para reforzar el aprendizaje y fomentar la práctica.
  - Evaluamos la comprensión de los conceptos clave mediante preguntas y respuestas.

Este capítulo es fundamental para entender cómo se extraen itemsets frecuentes de manera eficiente, sentando las bases para la generación de reglas de asociación y la aplicación de algoritmos más avanzados en capítulos posteriores.

---

## Referencias

- Agrawal, R., & Srikant, R. (1994). Fast algorithms for mining association rules. *Proceedings of the 20th International Conference on Very Large Data Bases*, 487-499.
- Han, J., Kamber, M., & Pei, J. (2011). *Data Mining: Concepts and Techniques*. Elsevier.
- Raschka, S. (2018). *MLxtend: Providing machine learning and data science utilities and extensions to Python’s scientific computing stack*. *The Journal of Open Source Software*, 3(24), 638.

---

## Próximos Pasos

En el siguiente capítulo, exploraremos la **Generación y Evaluación de Reglas de Asociación**, aprendiendo cómo convertir los itemsets frecuentes obtenidos mediante Apriori en reglas de asociación significativas y cómo evaluar su interés utilizando medidas como confianza, lift y otros.

---